In [1]:
import sys

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
    
import pandas as pd
import numpy as np
import ast

import tasks.run_helper

In [2]:
df = pd.read_csv(
    "../data/100_annotators.csv",
    converters={"annot_personality_characteristics": ast.literal_eval},
)
df["toxicity"] = df.annotation.apply(lambda x: x[-1]).astype(int)
df["annot_politics"] = df.annot_personality_characteristics.apply(
    lambda x: x[0]
)
df.annot_age = pd.cut(df.annot_age, bins=4)
df.message_id = df.message_id.astype(str)
df["comment_key"] = df.conv_id + df.message_id

df = df.loc[
    :,
    [
        "conv_id",
        "message_id",
        "comment_key",
        "message",
        "toxicity",
        "annot_age",
        "annot_sex",
        "annot_sexual_orientation",
        "annot_demographic_group",
        "annot_current_employment",
        "annot_education_level",
        "annot_politics",
    ],
]
df = df.groupby(["conv_id", "message_id", "comment_key", "message"]).apply(
    lambda x: pd.Series(
        {
            col: x[col].tolist()
            for col in df.columns
            if col not in ["conv_id", "message_id", "comment_key", "message"]
        }
    ),
    include_groups=False,
)
df["fake_index"] = 1
df.head(1)

,,,,toxicity,annot_age,annot_sex,annot_sexual_orientation,annot_demographic_group,annot_current_employment,annot_education_level,annot_politics,fake_index
conv_id,message_id,comment_key,message,,,,,,,,,
250cc245-86c4-4e97-a8ce-31d5d6796f71,-1428289379724132680,250cc245-86c4-4e97-a8ce-31d5d6796f71-1428289379724132680,"well thats a no brainer hillary clinton gave huma abdein a security clearance when she has ties to a known terrorist group the muslim brotherhood, and her mother runs an anti american news paper in the middle east, debbie washed up crook shultz got the awan famaily security clearances and they were recent immigrants, had absolutely no IT experience and possible ties to terrorist groups in pakistan. its pretty clear our liberal ran government is a complete and total failure when it comes to national security. 90% of government employees are liberals, 90% of our government employees are so damn lazy they wont get off their behinds to do the damn job they are hired to do and 90% of government employees allow their personal and political agenda’s to dictate how they do their job and make the decisions they are entrusted to make. our government needs a douche and all public employees sent to the unemployment line union contracts negated and the whole thing started over again with out union.\n","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[(69.75, 88.0], (33.25, 51.5], (69.75, 88.0], ...","[female, non-binary, female, male, male, male,...","[homosexual, straight, straight, straight, bis...","[black, white, asian, other, asian, other, asi...","[blue-collar, white-collar, unemployed, white-...","[university, none, university, none, universit...","[right-wing conservative, right-wing conservat...",1


In [3]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_age",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
"(51.5, 69.75]",0.014477,-0.004609,-0.001542,0.015298
"(14.927, 33.25]",-0.012664,0.015191,0.014854,-0.040938
"(69.75, 88.0]",0.006100,0.009411,0.003992,0.001733
"(33.25, 51.5]",-0.004004,-0.016691,-0.010497,0.028275


In [4]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_sex",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
non-binary,-0.011347,0.020129,-0.047526,-0.065838
male,0.001313,-0.001807,0.007015,0.026849
female,0.000671,-0.004702,0.002654,0.001559


In [5]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_demographic_group",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
other,-0.000170,0.021420,0.038209,0.013224
asian,-0.005055,0.003280,-0.033471,-0.001047
white,0.018281,-0.016823,-0.016615,-0.039265
black,-0.000291,-0.012082,-0.002108,-0.014922


In [6]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_sexual_orientation",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
homosexual,0.030096,-0.009012,-0.000344,-0.020070
straight,-0.003887,-0.002250,-0.006265,0.001182
bisexual,-0.000337,-0.002880,-0.046005,-0.032262
other,0.003770,0.019248,0.045700,0.037594


In [7]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_current_employment",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
unemployed,0.004161,-0.002923,-0.000120,-0.026297
white-collar,-0.002012,0.011295,-0.009451,0.018305
blue-collar,-0.005075,-0.012246,0.009299,0.006263


In [8]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_education_level",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
none,0.008815,0.025435,-0.005972,0.004447
university,-0.003958,-0.010567,-0.002554,0.000055
high-school,-0.002595,-0.008473,0.004877,-0.028332


In [9]:
tasks.run_helper.results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_politics",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/4 [00:00<?, ?it/s]

,250cc245-86c4-4e97-a8ce-31d5d6796f71,2feee653-15e9-4c38-b7a2-96fe26f50c9f,434ddba0-393a-424d-9c14-4837d454a221,a004f55d-03b1-4058-919a-6e6531aaf32f
left-wing liberal,0.001986,-0.009832,-0.013499,-0.009794
right-wing conservative,-0.006232,0.034614,-0.000024,-0.047970
apolitical,0.015690,0.002228,0.035095,0.019042


## Unified dataset

In [10]:
tasks.run_helper.results(
    df,
    discussion_id_col="fake_index",
    sdb_column="annot_age",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
"(51.5, 69.75]",0.004443
"(14.927, 33.25]",-0.004530
"(69.75, 88.0]",0.006843
"(33.25, 51.5]",-0.002460


In [11]:
tasks.run_helper.results(
    df,
    discussion_id_col="fake_index",
    sdb_column="annot_sex",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
non-binary,-0.026324
male,0.007125
female,0.000638


In [12]:
tasks.run_helper.results(
    df,
    discussion_id_col="fake_index",
    sdb_column="annot_demographic_group",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
other,0.018001
asian,-0.009256
white,-0.010836
black,-0.006948


In [13]:
tasks.run_helper.results(
    df,
    discussion_id_col="fake_index",
    sdb_column="annot_sexual_orientation",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
homosexual,0.000530
straight,-0.003252
bisexual,-0.016896
other,0.025422


In [14]:
tasks.run_helper.results(
    df,
    discussion_id_col="fake_index",
    sdb_column="annot_current_employment",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
unemployed,-0.006915
white-collar,0.005011
blue-collar,-0.000148


In [15]:
tasks.run_helper.results(
    df,
    discussion_id_col="fake_index",
    sdb_column="annot_education_level",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
none,0.009314
university,-0.004564
high-school,-0.008280


In [16]:
tasks.run_helper.results(
    df,
    discussion_id_col="fake_index",
    sdb_column="annot_politics",
    value_col="toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/1 [00:00<?, ?it/s]

,1
left-wing liberal,-0.007833
right-wing conservative,-0.003492
apolitical,0.017511
